In [1]:
import csv
import json

# Open the CSV file
csv_file = 'tarot_readings.csv'
json_file = 'tarot_readings.json'

# Read the CSV file and convert to dictionary
with open(csv_file, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    data = [row for row in reader]

# Write the dictionary to a JSON file
with open(json_file, mode='w', encoding='utf-8') as file:
    json.dump(data, file, indent=4)

print(f"CSV data has been converted to {json_file}")

CSV data has been converted to tarot_readings.json


In [2]:
import json

file = json.load(open("tarot_readings.json", "r"))
print(file[1])

{'Card 1': 'Three of Wands', ' Card 2': 'Death', ' Card 3': 'Queen of Pentacles', ' Reading': 'Your tarot reading is pointing to the need for transformation in order to move forward. The Three of Wands is a card of ambition and foresight, suggesting that the time is right to make a move or take on a new venture. Death is a card of change and transformation, and though it may seem like a scary card, it is a necessary step in order to experience growth. The Queen of Pentacles speaks of abundance and comfort, and is a reminder that you have the resources and support to make your move. With courage and determination, you can take the necessary steps to create the life you desire.'}


In [3]:
!pip install unsloth trl peft accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 4.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.1/376.1 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 872.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.

In [4]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU: Tesla T4


In [5]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/Phi-3-mini-4k-instruct-bnb-4bit"

max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.8: Fast Mistral patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [6]:
from datasets import Dataset
import json

# Strip any extra spaces from the column names
file = [dict((key.strip(), value) for key, value in item.items()) for item in file]

# Define a function to format the prompt correctly
def format_prompt(example):
    # Combine the three cards into an input string
    input_text = f"Cards: {example['Card 1']}, {example['Card 2']}, {example['Card 3']}"
    # The output is the "Reading" from your dataset
    output_text = example['Reading']

    # Return the formatted string
    return f"### Input: {input_text}\n### Output: {output_text}<|endoftext|>"

# Format the data
formatted_data = [format_prompt(item) for item in file]

# Create a dataset from the formatted data
dataset = Dataset.from_dict({"text": formatted_data})

# Print the first few entries to verify
print(dataset[0])

{'text': '### Input: Cards: The sun, Three of Cups, Two of Pentacles\n### Output: Your tarot reading speaks of joy, celebration, abundance and balance. The Sun card signifies success and accomplishment in the near future, a time to rejoice and share your happiness with those around you. The Three of Cups brings with it a feeling of joy, partying and social gathering, while the Two of Pentacles indicates that you are in a period of maintaining balance and managing your resources wisely. This is a time for you to enjoy the fruits of your labor and share them with those you love.<|endoftext|>'}


In [7]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank - higher = more capacity, more memory
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None, # LoftQ
)

Unsloth 2025.12.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Training arguments optimized for Unsloth
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # Effective batch size = 8
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
        report_to="none", # Disable Weights & Biases logging
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/5769 [00:00<?, ? examples/s]

In [9]:
# Train the model
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,769 | Num Epochs = 3 | Total steps = 2,166
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 119,537,664 of 3,940,617,216 (3.03% trained)


Step,Training Loss
25,1.268300
50,1.040300
75,1.004300
100,0.987700
125,0.975500
150,0.997900
175,0.979900
200,0.986600
225,0.964100
250,0.974800


Unsloth: Will smartly offload gradients to save VRAM!


In [10]:
# Simpler test prompt
messages = [
    {"role": "user", "content": "What is the meaning of the Three of Cups card?"}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)

response = tokenizer.batch_decode(outputs)[0]
print(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|user|> What is the meaning of the Three of Cups card?<|end|><|assistant|> The Three of Cups card is a symbol of joy, celebration, and friendship. It suggests that you are surrounded by people who care for you and will support you in times of need. The card encourages you to embrace the connections you have with others and to express your gratitude for their presence in your life. This card also indicates that you are likely to experience a period of contentment and happiness, where you can fully enjoy the company of your friends and loved ones.<|end|>


In [11]:
# Test 1
messages = [
    {"role": "user", "content": "If I draw The Fool, The Emperor, and The Hierophant for my career, what does it say?"}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)

response = tokenizer.batch_decode(outputs)[0]
print(response)

<|user|> If I draw The Fool, The Emperor, and The Hierophant for my career, what does it say?<|end|><|assistant|> The Fool symbolizes a new beginning, a fresh perspective, and a sense of optimism. The Emperor is a sign of structure, stability, and a commanding presence. The Hierophant is a representation of tradition, conformity, and spiritual guidance. If you draw these three symbols for your career, it suggests that you are at a crossroads where you need to balance your desire for freedom and exploration with the need for stability and security. You may need to find a way to honor both your inner and outer worlds, allowing you to move forward with confidence and clarity.<|end|>


In [12]:
# Test 2
messages = [
    {"role": "user", "content": "What does my career look like if I draw The Chariot, Knight of Pentacles, and Two of Wands?"}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)

response = tokenizer.batch_decode(outputs)[0]
print(response)

<|user|> What does my career look like if I draw The Chariot, Knight of Pentacles, and Two of Wands?<|end|><|assistant|> Your career is on an upward trajectory. The Chariot symbolizes forward motion and progress, while the Knight of Pentacles suggests that you are taking a practical and methodical approach to achieving your goals. The Two of Wands indicates that you have a strong vision for your future and are working hard to bring it to fruition. You are likely to experience success and recognition for your efforts, and your career will continue to grow and evolve.<|end|>


In [13]:
# Test 3
messages = [
    {"role": "user", "content": "What career challenges will I face with The Tower, Three of Swords, and Five of Pentacles?"}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)

response = tokenizer.batch_decode(outputs)[0]
print(response)

<|user|> What career challenges will I face with The Tower, Three of Swords, and Five of Pentacles?<|end|><|assistant|> The Tower, Three of Swords, and Five of Pentacles suggest that you may be facing a difficult time in your career. The Tower suggests that a sudden change is coming your way, and it may be a challenging one. The Three of Swords suggests that you may be experiencing some emotional pain and turmoil as a result of this change, but it is important to remember that it will pass. The Five of Pentacles suggests that you may be feeling financially insecure as a result of this change, but there are ways to overcome this and find stability again. It is important to stay focused and positive, and to remember that you have the strength and resilience to overcome any challenge.<|end|>


In [14]:
# Test 4
messages = [
    {"role": "user", "content": "What does my career growth look like with The Sun, Queen of Pentacles, and Ace of Wands?"}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)

response = tokenizer.batch_decode(outputs)[0]
print(response)

<|user|> What does my career growth look like with The Sun, Queen of Pentacles, and Ace of Wands?<|end|><|assistant|> The Sun in your career growth is a positive omen of success, prosperity, and joy. The Queen of Pentacles suggests that you will be able to manifest your dreams and desires into reality, and that you will be able to enjoy the fruits of your hard work. The Ace of Wands is a sign of new beginnings, creativity, and passion. It suggests that you will be able to start fresh and that you will have the energy and enthusiasm to pursue your goals. Overall, your career growth looks very positive and filled with potential.<|end|>


In [15]:
# Test 5
messages = [
    {"role": "user", "content": "What advice does The Star, Eight of Pentacles, and King of Swords give me for my career?"}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)

response = tokenizer.batch_decode(outputs)[0]
print(response)

<|user|> What advice does The Star, Eight of Pentacles, and King of Swords give me for my career?<|end|><|assistant|> The Star symbolizes hope and optimism, and encourages you to have faith in your own abilities and in the universe. The Eight of Pentacles suggests that you are in the process of mastering a skill or learning something new, and that your hard work will pay off. The King of Swords encourages you to think logically and to make decisions based on facts and rational thinking. You have the potential to achieve great success in your career, but remember to stay grounded and to use your intuition and intellect to guide you.<|end|>


In [16]:
# Test 6
messages = [
    {"role": "user", "content": "What does my immediate career outlook look like with The World, Ten of Cups, and The Magician?"}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)

response = tokenizer.batch_decode(outputs)[0]
print(response)

<|user|> What does my immediate career outlook look like with The World, Ten of Cups, and The Magician?<|end|><|assistant|> Your immediate career outlook is very positive with the World, Ten of Cups, and The Magician. The World card is a sign of completion and success, and it suggests that you have achieved a great deal in your career and are now ready to move on to the next level. The Ten of Cups indicates that you have a strong support system and that your family and friends are behind you in your endeavors. Finally, The Magician is a card of creativity and resourcefulness, and it suggests that you have the skills and knowledge necessary to take your career to the next level. With all of these cards together, you have the potential to achieve great success in your career.<|end|>


In [17]:
import gc, torch

# If you still have these objects from training, delete them:
for name in ["trainer", "trainer_stats", "outputs", "inputs", "dataset"]:
    if name in globals():
        del globals()[name]

gc.collect()
torch.cuda.empty_cache()

In [18]:
model.eval()
model = model.to("cpu")
torch.cuda.empty_cache()
gc.collect()

0

In [19]:
import gc, torch

for name in [
    "file", "data", "formatted_data", "dataset",
    "trainer", "trainer_stats",
    "inputs", "outputs",
]:
    if name in globals():
        del globals()[name]

gc.collect()
torch.cuda.empty_cache()

In [20]:
from google.colab import drive
drive.mount("/content/drive")

SAVE_DIR = "/content/drive/MyDrive/phi3_tarot_model"

model.save_pretrained(SAVE_DIR)          # saves LoRA adapter if model is PEFT
tokenizer.save_pretrained(SAVE_DIR)

print("Saved to:", SAVE_DIR)

Mounted at /content/drive
Saved to: /content/drive/MyDrive/phi3_tarot_model


In [1]:
from unsloth import FastLanguageModel
from peft import PeftModel

base_name = "unsloth/Phi-3-mini-4k-instruct-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=base_name,
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

ADAPTER_DIR = "/content/drive/MyDrive/phi3_tarot_model"
model = PeftModel.from_pretrained(model, ADAPTER_DIR)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.8: Fast Mistral patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [2]:
import os
print(os.listdir("/content/drive/MyDrive/phi3_tarot_model"))

['README.md', 'adapter_model.safetensors', 'adapter_config.json', 'chat_template.jinja', 'tokenizer_config.json', 'special_tokens_map.json', 'added_tokens.json', 'tokenizer.model', 'tokenizer.json']


In [3]:
print(type(model))
print(model.peft_config)

<class 'peft.peft_model.PeftModelForCausalLM'>
{'default': LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping={'base_model_class': 'MistralForCausalLM', 'parent_library': 'transformers.models.mistral.modeling_mistral', 'unsloth_fixed': True}, peft_version='0.18.0', base_model_name_or_path='unsloth/Phi-3-mini-4k-instruct-bnb-4bit', revision=None, inference_mode=True, r=64, target_modules={'q_proj', 'up_proj', 'down_proj', 'o_proj', 'k_proj', 'gate_proj', 'v_proj'}, exclude_modules=None, lora_alpha=128, lora_dropout=0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, alora_invocation_tokens=None, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=Lora

In [4]:
%%bash
set -e
cd /content

rm -rf llama.cpp
git clone https://github.com/ggml-org/llama.cpp
cmake llama.cpp -B llama.cpp/build -DBUILD_SHARED_LIBS=OFF
# Build with 1 job to avoid RAM spikes
cmake --build llama.cpp/build --config Release -j 1 --target llama-quantize llama-gguf-split
cp llama.cpp/build/bin/llama-* llama.cpp/ || true
ls -la llama.cpp/llama-*

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Git: /usr/bin/git (found version "2.34.1")
-- The ASM compiler identification is GNU
-- Found assembler: /usr/bin/cc
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- GGML_SYSTEM_ARCH: x86
-- Including CPU backend
-- Found OpenMP_C: -fopenmp (found version "

Cloning into 'llama.cpp'...
Updating files: 100% (2084/2084), done.
CMAKE_BUILD_TYPE=Release


In [6]:
import os

out_dir = "/content/drive/MyDrive/gguf_model"
os.makedirs(out_dir, exist_ok=True)
print("Exists:", os.path.exists(out_dir), " -> ", out_dir)

Exists: True  ->  /content/drive/MyDrive/gguf_model


In [8]:
HF_DIR = "/content/drive/MyDrive/phi3_tarot_hf_merged"
import os
os.makedirs(HF_DIR, exist_ok=True)

# If model is still a PeftModel, merge first:
from peft import PeftModel
if isinstance(model, PeftModel):
    model = model.merge_and_unload()

model.save_pretrained(HF_DIR)          # creates config.json + weights shards
tokenizer.save_pretrained(HF_DIR)

print("Saved HF model to:", HF_DIR)

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Saved HF model to: /content/drive/MyDrive/phi3_tarot_hf_merged


In [10]:
import os, glob
HF_DIR = "/content/drive/MyDrive/phi3_tarot_hf_merged"

print("Has config:", os.path.exists(HF_DIR + "/config.json"))
print("Weight shards:", glob.glob(HF_DIR + "/*.safetensors")[:5])
print("Index:", os.path.exists(HF_DIR + "/model.safetensors.index.json") or
              os.path.exists(HF_DIR + "/pytorch_model.bin.index.json"))

Has config: True
Weight shards: ['/content/drive/MyDrive/phi3_tarot_hf_merged/model.safetensors']
Index: False


In [11]:
import shutil, os

SRC = "/content/drive/MyDrive/phi3_tarot_hf_merged"
DST = "/content/phi3_tarot_hf_merged"

if os.path.exists(DST):
    shutil.rmtree(DST)

shutil.copytree(SRC, DST)
print("Copied to:", DST, "files:", os.listdir(DST))

Copied to: /content/phi3_tarot_hf_merged files: ['special_tokens_map.json', 'added_tokens.json', 'tokenizer_config.json', 'config.json', 'chat_template.jinja', 'model.safetensors', 'tokenizer.model', 'tokenizer.json', 'generation_config.json']


In [13]:
%%bash
ls -lah /content | sed -n '1,120p'
echo "---- GGUF files in /content ----"
ls -lah /content/*.gguf 2>/dev/null || echo "No .gguf in /content"

total 6.4M
drwxr-xr-x  1 root root 4.0K Dec 22 13:11 .
drwxr-xr-x  1 root root 4.0K Dec 22 10:05 ..
drwxr-xr-x  4 root root 4.0K Dec  9 14:41 .config
drwx------  5 root root 4.0K Dec 22 12:23 drive
drwxr-xr-x  4 root root 4.0K Dec 22 10:23 huggingface_tokenizers_cache
drwxr-xr-x 27 root root 4.0K Dec 22 12:53 llama.cpp
drwxr-xr-x  4 root root 4.0K Dec 22 12:07 outputs
drwx------  2 root root 4.0K Dec 22 12:57 phi3_tarot_hf_merged
drwxr-xr-x  1 root root 4.0K Dec  9 14:42 sample_data
-rw-r--r--  1 root root 2.9M Dec 22 10:16 tarot_readings.csv
-rw-r--r--  1 root root 3.5M Dec 22 10:16 tarot_readings.json
drwxr-xr-x  4 root root 4.0K Dec 22 10:22 unsloth_compiled_cache
---- GGUF files in /content ----
No .gguf in /content


In [14]:
%%bash
echo "---- GGUF files in model folder ----"
ls -lah /content/phi3_tarot_hf_merged/*.gguf 2>/dev/null || echo "No .gguf in /content/phi3_tarot_hf_merged"

---- GGUF files in model folder ----
No .gguf in /content/phi3_tarot_hf_merged


In [15]:
import subprocess, textwrap

cmd = (
    "python /content/llama.cpp/unsloth_convert_hf_to_gguf.py "
    "--outfile /content/phi3_tarot.F16.gguf "
    "--outtype f16 "
    "--split-max-size 50G "
    "/content/phi3_tarot_hf_merged"
)

p = subprocess.run(cmd, shell=True, text=True, capture_output=True)

print("Return code:", p.returncode)
print("\n--- STDERR (tail) ---\n", p.stderr[-4000:])
print("\n--- STDOUT (tail) ---\n", p.stdout[-2000:])

Return code: 1

--- STDERR (tail) ---
 INFO:hf-to-gguf:Loading model: phi3_tarot_hf_merged
INFO:hf-to-gguf:Model architecture: MistralForCausalLM
INFO:hf-to-gguf:gguf: indexing model part 'model.safetensors'
Traceback (most recent call last):
  File "/content/llama.cpp/unsloth_convert_hf_to_gguf.py", line 11094, in <module>
    main()
  File "/content/llama.cpp/unsloth_convert_hf_to_gguf.py", line 11071, in main
    model_instance = model_class(dir_model, output_type, fname_out,
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/llama.cpp/unsloth_convert_hf_to_gguf.py", line 2709, in __init__
    super().__init__(*args, **kwargs)
  File "/content/llama.cpp/unsloth_convert_hf_to_gguf.py", line 1054, in __init__
    super().__init__(*args, **kwargs)
  File "/content/llama.cpp/unsloth_convert_hf_to_gguf.py", line 454, in __init__
    self.dequant_model()
  File "/content/llama.cpp/unsloth_convert_hf_to_gguf.py", line 763, in dequant_model
    raise NotImp

In [16]:
%%bash
ls -lah /content/phi3_tarot_hf_merged | sed -n '1,200p'

total 2.2G
drwx------ 2 root root 4.0K Dec 22 12:57 .
drwxr-xr-x 1 root root 4.0K Dec 22 13:11 ..
-rw------- 1 root root  293 Dec 22 12:57 added_tokens.json
-rw------- 1 root root  407 Dec 22 12:57 chat_template.jinja
-rw------- 1 root root 1.1K Dec 22 12:56 config.json
-rw------- 1 root root  194 Dec 22 12:56 generation_config.json
-rw------- 1 root root 2.2G Dec 22 12:57 model.safetensors
-rw------- 1 root root  572 Dec 22 12:57 special_tokens_map.json
-rw------- 1 root root 2.9K Dec 22 12:57 tokenizer_config.json
-rw------- 1 root root 3.5M Dec 22 12:57 tokenizer.json
-rw------- 1 root root 489K Dec 22 12:57 tokenizer.model


In [17]:
from unsloth import FastLanguageModel
from peft import PeftModel
import torch

# IMPORTANT: NOT the bnb-4bit one
base_name = "unsloth/Phi-3-mini-4k-instruct"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=base_name,
    max_seq_length=2048,
    dtype=torch.float16,     # or torch.bfloat16 if available
    load_in_4bit=False,
)

model = PeftModel.from_pretrained(model, "/content/drive/MyDrive/phi3_tarot_model")

==((====))==  Unsloth 2025.12.8: Fast Mistral patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.65G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [18]:
model = model.merge_and_unload()

In [19]:
import os
HF16_DIR = "/content/phi3_tarot_hf16_merged"
os.makedirs(HF16_DIR, exist_ok=True)

model.save_pretrained(HF16_DIR, safe_serialization=True)
tokenizer.save_pretrained(HF16_DIR)

print("Saved merged fp16 HF model to:", HF16_DIR)

Saved merged fp16 HF model to: /content/phi3_tarot_hf16_merged


In [20]:
import subprocess

cmd = (
    "python /content/llama.cpp/unsloth_convert_hf_to_gguf.py "
    "--outfile /content/phi3_tarot.F16.gguf "
    "--outtype f16 "
    "--split-max-size 50G "
    f"{HF16_DIR}"
)

p = subprocess.run(cmd, shell=True, text=True)
print("Return code:", p.returncode)

Return code: 0


In [21]:
import os
print("F16 exists:", os.path.exists("/content/phi3_tarot.F16.gguf"))

F16 exists: True


In [22]:
%%bash
set -e
cd /content
./llama.cpp/llama-quantize /content/phi3_tarot.F16.gguf /content/phi3_tarot.Q4_K_M.gguf q4_k_m
ls -lah /content/phi3_tarot.Q4_K_M.gguf


main: quantize time = 411482.07 ms
main:    total time = 411482.07 ms
-rw-r--r-- 1 root root 2.2G Dec 22 13:50 /content/phi3_tarot.Q4_K_M.gguf


main: build = 7507 (3997c78e3)
main: built with GNU 11.4.0 for Linux x86_64
main: quantizing '/content/phi3_tarot.F16.gguf' to '/content/phi3_tarot.Q4_K_M.gguf' as Q4_K_M
llama_model_loader: loaded meta data with 34 key-value pairs and 291 tensors from /content/phi3_tarot.F16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Phi3_Tarot_Hf16_Merged
llama_model_loader: - kv   3:                       general.quantized_by str              = Unsloth
llama_model_loader: - kv   4:                         general.size_label str              = 3.8B
llama_model_loader: - kv   5:                           general.repo_url str         

In [23]:
import os, shutil
OUT = "/content/drive/MyDrive/phi3_tarot_gguf"
os.makedirs(OUT, exist_ok=True)

shutil.move("/content/phi3_tarot.Q4_K_M.gguf", OUT + "/phi3_tarot.Q4_K_M.gguf")
print("GGUF saved to:", OUT + "/phi3_tarot.Q4_K_M.gguf")

GGUF saved to: /content/drive/MyDrive/phi3_tarot_gguf/phi3_tarot.Q4_K_M.gguf
